In [1]:
# load iamges

from pathlib import Path

data_dir = Path('./data/wikiart/')


# count images
cnt = 0
for dir in data_dir.iterdir():
    if dir.is_dir():
        for file in dir.iterdir():
            if str(file).endswith('jpg'):
                cnt+=1
cnt

81446

In [2]:
import pandas as pd
# load captions data
df=pd.read_csv('/Users/simon/Downloads/official_data/artemis_dataset_release_v0.csv')

In [3]:
df.columns

Index(['art_style', 'painting', 'emotion', 'utterance', 'repetition'], dtype='object')

In [8]:
def add_captions(df,artist_name):
    #get 'art_style' value
    style=row['art_style'].replace('_',' ').lower()
    style=style[0].upper()+style[1:]
    df['caption']=df.apply(lambda row:f"{style} artwork by {artist_name}. The painting expresses {row['emotion']} because {row['utterance'].lower()}",axis=1)
    return df

In [9]:
import json
import shutil
from PIL import Image,ImageFile
from tqdm import tqdm
from pathlib import Path
ImageFile.LOAD_TRUNCATED_IMAGES = True


save_dir='./data/artemis-laion/'
shutil.rmtree(save_dir, ignore_errors=True)
save_dir=Path(save_dir).absolute()
save_dir.mkdir(parents=True,exist_ok=True)
#remove save_dir and content if it exists

iter=0
for style_dir in tqdm(list(data_dir.iterdir())):
    if style_dir.is_dir():
        for img_file in style_dir.glob('*.jpg'):

            #IMAGES
            img = Image.open(img_file)
            width,height = img.size
            if height < 64 or width < 64:
                continue
            elif height > 512 and width > 512:
                #change largest dimension to 512 and keep aspect ratio
                if height > width:
                    resized_img = img.resize((512, int(512*height/width)), Image.ANTIALIAS)
                    resized_img.save(save_dir/str(str(iter)+'.jpg'))

                else:
                    resized_img = img.resize((int(512*width/height), 512), Image.ANTIALIAS)
                    resized_img.save(save_dir/str(str(iter)+'.jpg'))

            elif height > 512:
                #resize to 512 and keep aspect ratio
                resized_img = img.resize((512, int(512*height/width)), Image.ANTIALIAS)
                #save image as resized
                resized_img.save(save_dir/str(str(iter)+'.jpg'))
            elif width > 512:
                #resize to 512 and keep aspect ratio
                resized_img = img.resize((int(512*width/height), 512), Image.ANTIALIAS)
                #save image as resized
                resized_img.save(save_dir/str(str(iter)+'.jpg'))
            else:    
                img.save(save_dir/str(str(iter)+'.jpg'))
            
            
            #CREATE CAPTION

            #match filename and with df row
            painting_name=img_file.name.replace('.jpg','')
            df_match=df[df['painting']==painting_name]
            if len(df_match)==0:
                continue
            artist_name=str(img_file.name).split('_')[0]
            artist_name=artist_name.replace('-', ' ')
            artist_name=artist_name.title()
            df_caption=add_captions(df_match,artist_name)
            for i,row in df_caption.iterrows():
                #save caption
                with open(save_dir/str(str(iter)+'.txt'),'w+') as f:
                    f.write(row['caption'])
                
                #CREATE METADATA
                if resized_img:
                    width,height = resized_img.size
                
                meta_data={'url':'https://www.artemisdataset.org/','width':width,'height':height}
                #save metadata
                with open(save_dir/str(str(iter)+'.json'),'w+') as f:
                    f.write(json.dumps(meta_data))
                iter+=1



  0%|          | 0/65 [00:00<?, ?it/s]/var/folders/g8/g2fn4cds1vj2yp9yzsd9158r0000gn/T/ipykernel_75099/291680148.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['caption']=df.apply(lambda row:f"{style} artwork by {artist_name}. The painting expresses {row['emotion']} because {row['utterance'].lower()}",axis=1)
 55%|█████▌    | 36/65 [21:13<12:19, 25.48s/it] /Users/simon/miniforge3/envs/target-screener/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/simon/miniforge3/envs/target-screener/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (107327830 pixels) exc